# Z-Score Normalization

[]

In [57]:
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split

DATA_PATH = './data'
ratings = pd.read_csv(f'{DATA_PATH}/ratings.csv')
movies  = pd.read_csv(f'{DATA_PATH}/movies.csv')

#SEED_LIST = [10, 21, 35, 42, 57, 60, 73, 88, 95, 101] -> 5개로 감소소
SEED_LIST = [10, 21, 35, 42, 57]
PERSONA_HEAVY_USER = 414
PERSONA_GENRE_SPECIALIST = 85

## 1. 전처리 함수 (Z-Score + 최소 평점 필터링)

In [58]:
def run_preprocessing(train_df, test_df):
    train = train_df.copy()
    test  = test_df.copy()

    # 1. 최소 5회 이상 평점 준 사용자만 사용 (페르소나는 강제 포함)
    user_counts = train['userId'].value_counts()
    valid_users = set(user_counts[user_counts >= 5].index)
    valid_users = valid_users.union({PERSONA_HEAVY_USER, PERSONA_GENRE_SPECIALIST})

    train = train[train['userId'].isin(valid_users)]
    test  = test[test['userId'].isin(valid_users)]

    # 2. Train에서만 사용자별 mean / std 계산
    user_stats = train.groupby('userId')['rating'].agg(['mean', 'std']).fillna(1.0)
    user_stats['std'] = user_stats['std'].replace(0, 1.0)   # std=0 방지

    # 3. Z-Score 변환 (Train)
    train = train.merge(user_stats, on='userId', suffixes=('', '_user'))
    train['z_rating'] = (train['rating'] - train['mean']) / train['std']

    # 4. Test도 동일한 통계량으로 변환 (Train에 없는 사용자는 제거 → cold-start 방지)
    test = test.merge(user_stats, on='userId', how='left')
    test = test.dropna(subset=['mean', 'std'])   # Train에 없는 사용자 제거
    test['z_rating'] = (test['rating'] - test['mean']) / test['std']

    return train, test, user_stats   # user_stats도 반환 (역변환에 필요)

## 2. 평가 
Significance Weighting 추가가

In [59]:
def apply_significance_weighting(R, sim_matrix, threshold=2):
    """
    Z-Score 기반 유사도 행렬에 Significance Weighting을 적용합니다.
    Significance Weighting: 공통 평가자 수가 임계값보다 적으면 유사도 값을 감쇠시킵니다.

    Args:
        R (np.array): 사용자-아이템 행렬 (Z-Score 적용).
        sim_matrix (np.array): 아이템 간 유사도 행렬 (R.T @ R).
        threshold (int): 공통 평가자 수 임계값 (예: 50).

    Returns:
        np.array: 유의성 가중치가 적용된 유사도 행렬.
    """
    # 1. 공통 평가자 수 (Co-Count) 행렬 C 계산
    # R이 0이 아닌 곳은 1로, 0인 곳은 0으로 변환 (평가 여부)
    R_binary = (R != 0).astype(float)
    # C = R_binary.T @ R_binary: 두 아이템을 모두 평가한 사용자 수를 계산
    co_count_matrix = np.dot(R_binary.T, R_binary)

    # 2. Significance Weighting Factor SW 계산
    # min(CoCount / threshold, 1)을 계산하여 가중치 행렬 생성
    sw_factor = np.minimum(co_count_matrix / threshold, 1.0)
    
    # 3. 유의성 가중치 적용
    # S_sw = S * SW (유사도 * 가중치)
    weighted_sim_matrix = sim_matrix * sw_factor
    
    return weighted_sim_matrix


In [60]:
import numpy as np
import time
# 수정된 fast_predict_matrix_no_k 함수 (Significance Weighting 적용)
def fast_predict_matrix_no_k(train_df):
    """
    K 제한 없이, 유사도가 양수인 모든 아이템을 사용하여 예측 (SW 적용)
    """
    # 1. Pivot Table 생성 및 R 행렬 정의
    pivot_df = train_df.pivot(index='userId', columns='movieId', values='z_rating').fillna(0)
    R = pivot_df.values  # (Users, Items)
    user_ids = pivot_df.index
    movie_ids = pivot_df.columns
    
    # 2. 아이템 간 유사도 행렬 계산 (Z-Score 내적)
    sim_matrix = np.dot(R.T, R)
    np.fill_diagonal(sim_matrix, -np.inf)
    
    # [핵심 변경 1] Significance Weighting 적용 (임계값 50 설정)
    sim_matrix_sw = apply_significance_weighting(R, sim_matrix, threshold=2)

    # 3. Top-K 필터링 로직 삭제! 양수 유사도만 남김 (SW 적용된 유사도 사용)
    # S는 양수 유사도만 포함하며, SW가 적용된 값임.
    S = np.where(sim_matrix_sw > 0, sim_matrix_sw, 0) 

    # 4. 예측 평점 계산 (Weighted Sum) -> 분자
    numerator = np.dot(R, S)
    
    # 5. 가중치 합 계산 (Sum of Weights) -> 분모
    # 분모는 예측 평점 계산에 사용된 가중치(S)의 절대값의 합으로 계산해야 함.
    R_binary = (R != 0).astype(float)
    denominator = np.dot(R_binary, np.abs(S))
    
    # 6. 나눗셈 (0으로 나누기 방지)
    with np.errstate(divide='ignore', invalid='ignore'):
        pred_z = numerator / denominator
        pred_z = np.nan_to_num(pred_z)
        
    return pred_z, user_ids, movie_ids

In [61]:
def rmse(y_true, y_pred): return np.sqrt(np.mean((np.array(y_true)-np.array(y_pred))**2))
def mae(y_true, y_pred):  return np.mean(np.abs(np.array(y_true)-np.array(y_pred)))

def time_predictions(fn, pairs):
    s = time.perf_counter()
    preds = [fn(u,i) for u,i in pairs]
    e = time.perf_counter() - s
    return preds, e, (e/len(pairs))*1000

## 3. Multi-seed 실험 루프

In [62]:
# ==========================================
# [3. Multi-seed 실험 루프 - K 제한 없음 버전]
# ==========================================

results = []
best_reco = None

print(f"실험 시작: K 제한 없이 모든 양수 유사도 사용. 총 {len(SEED_LIST)}개의 시드 테스트.")

for idx, SEED in enumerate(SEED_LIST):
    print(f"\nRunning Seed {SEED} ({idx+1}/{len(SEED_LIST)})")

    # 1. 데이터 분할 및 전처리
    train_raw, test_raw = train_test_split(ratings, test_size=0.2, random_state=SEED)
    train, test, user_stats = run_preprocessing(train_raw, test_raw)

    # 2. [핵심] K 제한 없는 행렬 연산으로 모든 예측값 한 번에 계산
    s = time.perf_counter()
    pred_matrix_z, u_ids, m_ids = fast_predict_matrix_no_k(train) # K 파라미터가 사라짐
    e = time.perf_counter() - s
    
    # 인덱싱 맵핑
    u_map = {u: i for i, u in enumerate(u_ids)}
    m_map = {m: i for i, m in enumerate(m_ids)}

    # 3. Test 샘플링 (평가용 데이터 3000개)
    rng = np.random.default_rng(SEED)
    test_sample = test[['userId', 'movieId', 'rating', 'mean', 'std']].sample(n=min(3000, len(test)), random_state=SEED)

    # 4. Test 데이터에 대한 예측값 추출
    z_preds = []
    for uid, mid in zip(test_sample['userId'], test_sample['movieId']):
        if uid in u_map and mid in m_map:
            u_idx = u_map[uid]
            m_idx = m_map[mid]
            z_preds.append(pred_matrix_z[u_idx, m_idx])
        else:
            z_preds.append(0.0)

    # 5. 역변환
    original_preds = []
    for pred_z, row in zip(z_preds, test_sample.itertuples()):
        mean_u = row.mean
        std_u  = row.std
        original_preds.append(pred_z * std_u + mean_u)

    # 6. 결과 저장
    avg_ms = (e / len(test_sample)) * 1000 
    
    res = {
        'seed': SEED,
        'rmse': rmse(test_sample['rating'], original_preds),
        'mae' : mae(test_sample['rating'], original_preds),
        'avg_ms': avg_ms
    }
    results.append(res)
    print(f" -> RMSE: {res['rmse']:.4f}, Time: {e:.4f}s")

    # 7. 첫 번째 시드(Seed 10)에서만 추천 결과 생성 (로직 동일)
    if idx == 0:
        def recommend(user_id, topk=3):
            if user_id not in u_map: return None
            u_idx = u_map[user_id]
            user_row_preds = pred_matrix_z[u_idx] 
            watched = set(train[train['userId']==user_id]['movieId'])
            
            candidates = []
            for m_idx, m_id in enumerate(m_ids):
                if m_id in watched: continue
                
                z = user_row_preds[m_idx]
                mean_u = user_stats.loc[user_id, 'mean']
                std_u  = user_stats.loc[user_id, 'std']
                final_score = z * std_u + mean_u
                
                candidates.append((m_id, final_score))
            
            candidates.sort(key=lambda x: -x[1])
            top = candidates[:topk]
            
            return [{
                'movieId': m,
                'title': movies.loc[movies['movieId']==m, 'title'].iloc[0] if len(movies.loc[movies['movieId']==m]) > 0 else "Unknown",
                'predicted_rating': round(score, 3)
            } for m, score in top]

        print("\n[추천 결과 생성 중...]")
        best_reco = {
            'heavy': recommend(PERSONA_HEAVY_USER),
            'specialist': recommend(PERSONA_GENRE_SPECIALIST)
        }

print("\n실험 완료! (결과 요약 셀을 실행하세요)")

실험 시작: K 제한 없이 모든 양수 유사도 사용. 총 5개의 시드 테스트.

Running Seed 10 (1/5)


C:\Users\dlxod\AppData\Local\Temp\ipykernel_28340\1065311077.py:26: RuntimeWarning: invalid value encountered in multiply
  weighted_sim_matrix = sim_matrix * sw_factor


 -> RMSE: 0.8614, Time: 7.5756s

[추천 결과 생성 중...]

Running Seed 21 (2/5)


C:\Users\dlxod\AppData\Local\Temp\ipykernel_28340\1065311077.py:26: RuntimeWarning: invalid value encountered in multiply
  weighted_sim_matrix = sim_matrix * sw_factor


 -> RMSE: 0.8660, Time: 5.2041s

Running Seed 35 (3/5)


...

 -> RMSE: 0.8747, Time: 4.8793s

Running Seed 42 (4/5)


C:\Users\dlxod\AppData\Local\Temp\ipykernel_28340\1065311077.py:26: RuntimeWarning: invalid value encountered in multiply
  weighted_sim_matrix = sim_matrix * sw_factor


 -> RMSE: 0.8908, Time: 4.1030s

Running Seed 57 (5/5)


C:\Users\dlxod\AppData\Local\Temp\ipykernel_28340\1065311077.py:26: RuntimeWarning: invalid value encountered in multiply
  weighted_sim_matrix = sim_matrix * sw_factor


 -> RMSE: 0.8411, Time: 4.5130s

실험 완료! (결과 요약 셀을 실행하세요)


## 4. 결과 요약

In [63]:
import pandas as pd
df = pd.DataFrame(results)
print(df[['seed','rmse','mae','avg_ms']].round(4))
print("\n=== 평균 ± 표준편차 ===")
print(f"RMSE : {df.rmse.mean():.4f} ± {df.rmse.std():.4f}")
print(f"MAE  : {df.mae.mean():.4f} ± {df.mae.std():.4f}")
print(f"예측 속도 : {df.avg_ms.mean():.2f} ms (± {df.avg_ms.std():.2f})")

print("\n=== 페르소나 추천 결과 (Seed 10) ===")
print(f"헤비 유저 (userId={PERSONA_HEAVY_USER})")
for x in best_reco['heavy']:
    print(f"  → {x['title']}  (예측 평점 {x['predicted_rating']})")

print(f"\n드라마 전문가 (userId={PERSONA_GENRE_SPECIALIST})")
for x in best_reco['specialist']:
    print(f"  → {x['title']}  (예측 평점 {x['predicted_rating']})")

   seed    rmse     mae  avg_ms
0    10  0.8614  0.6494  2.5252
1    21  0.8660  0.6591  1.7347
2    35  0.8747  0.6650  1.6264
3    42  0.8908  0.6723  1.3677
4    57  0.8411  0.6413  1.5043

=== 평균 ± 표준편차 ===
RMSE : 0.8668 ± 0.0182
MAE  : 0.6574 ± 0.0123
예측 속도 : 1.75 ms (± 0.45)

=== 페르소나 추천 결과 (Seed 10) ===
헤비 유저 (userId=414)
  → Harmonists, The (1997)  (예측 평점 5.0)
  → Taxi 3 (2003)  (예측 평점 4.945)
  → The Red Turtle (2016)  (예측 평점 4.804)

드라마 전문가 (userId=85)
  → Mr. Wrong (1996)  (예측 평점 5.0)
  → Before and After (1996)  (예측 평점 5.0)
  → Awfully Big Adventure, An (1995)  (예측 평점 5.0)
